# NGEN Simulation Data Query Examples
This notebook provides examples of querying and visualizing some NGEN data that has been transformed and stored in the TEEHR format.

In [ ]:
# Import the required packages.
import holoviews as hv
import geoviews as gv
from pathlib import Path
import hvplot.pandas
import cartopy.crs as ccrs
from holoviews import opts
import duckdb
import geopandas as gpd

import teehr.queries.duckdb as tqd

In [ ]:
# Set some notebook variables to point to the relevant study files.
STUDY_DIR = Path(Path.home(), "shared/rti-eval/ngen-simulation-example/")
STUDY_TS_DIR = Path(STUDY_DIR, "timeseries")
STUDY_GEO_DIR = Path(STUDY_DIR, "geo")
PRIMARY_FILEPATH = Path(STUDY_TS_DIR, "usgs.parquet")
SECONDARY_FILEPATH = Path(STUDY_TS_DIR, "nexus_simulation.parquet")
CROSSWALK_FILEPATH = Path(STUDY_GEO_DIR, "usgs_nex_crosswalk.parquet")
GEOMETRY_FILEPATH = Path(STUDY_GEO_DIR, "usgs_geometry.parquet")

CATCHMENT_FILEPATH = Path(STUDY_GEO_DIR, "cat_geometry.parquet")
NEXUS_FILEPATH = Path(STUDY_GEO_DIR, "nex_geometry.parquet")

In [ ]:
catchments_gdf = catchments = gpd.read_parquet(CATCHMENT_FILEPATH).to_crs("EPSG:3857")
catchments = catchments_gdf.hvplot(color="lightblue")

In [ ]:
nexus_gdf = gpd.read_parquet(NEXUS_FILEPATH).to_crs("EPSG:3857")
nexus = nexus_gdf.hvplot(color="green", size=30)

In [ ]:
tiles = gv.tile_sources.OSM
(tiles * catchments * nexus).opts(width=600, height=600, show_legend=False)

In [ ]:
query_gdf = tqd.get_metrics(
    primary_filepath=PRIMARY_FILEPATH,
    secondary_filepath=SECONDARY_FILEPATH,
    crosswalk_filepath=CROSSWALK_FILEPATH,
    geometry_filepath=GEOMETRY_FILEPATH,
    group_by=["primary_location_id"],
    order_by=["primary_location_id"],
    return_query=False,
    include_geometry=True,
    include_metrics=["bias"]
)

In [ ]:
query_gdf

In [ ]:
%%time
query_gdf_prj = query_gdf.to_crs("EPSG:3857")

In [ ]:
query_hvplot = query_gdf_prj.hvplot(
    # color="red",
    crs=ccrs.GOOGLE_MERCATOR,
    hover_cols=["primary_location_id", "bias"],
    c="bias",
    size=100
)
(tiles * catchments * nexus * query_hvplot).opts(width=600, height=600, show_legend=False)

In [ ]:
%%time
query_df = tqd.get_joined_timeseries(
        primary_filepath=PRIMARY_FILEPATH,
        secondary_filepath=SECONDARY_FILEPATH,
        crosswalk_filepath=CROSSWALK_FILEPATH,
        geometry_filepath=GEOMETRY_FILEPATH,
        order_by=["primary_location_id"],
        filters=[{
            "column": "primary_location_id",
            "operator": "=",
            "value": "usgs-02450180"

        }],
        return_query=False,
)

In [ ]:
query_df

In [ ]:
obs = query_df.hvplot("value_time", "primary_value", color="blue")
sim = query_df.hvplot("value_time", "secondary_value", color="red")
obs * sim